In [14]:
include("optim_lib.jl")
include("lib_func.jl")
using Optim, DataFrames, CSV

# Load the dataframe from the CSV file
df_2d = CSV.read("data/camel_d6.csv", DataFrame)


# Constants and Parameters
d1, d2, ds = 8, 8, 1  # Degree range and step
const n, a, b = 4, 5, 1 
const C = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alph = .9 , 2 / 10  # Sampling parameters
f = camel_4d # Objective function


camel_4d (generic function with 1 method)

In [15]:
coeffs_poly_approx = main_gen(f, n, d1, d2, ds, delta, alph, C, 0.2)

dimension Vector space: 495
sample size: 2401


1-element Vector{Vector{Float64}}:
 [910.0459729904072, 1408.7984653708456, -1037.462378818986, 1376.2055558825919, 625.7360132503313, -700.5096451916586, 326.84052975595927, -1.9649100368298486e18, -1186.8388788939176, 19.495898623965143  …  1.7591330308966363e-12, -749.590170187614, 4.0961378501417184e-12, 7.165264514435251e-13, 8.528133754185396e-13, -7.800751716306971e17, -7.09718547170082e17, 2.8673883600785597e17, 1.0902779942033289e18, 844.6032699677081]

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [16]:
using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, ProgressLogging, DataFrames, Optim

include("hom_solve.jl") # Include the homotopy solver and main function
vars = @polyvar(x[1:n]) # Define polynomial ring 
h_x, h_y, h_z, h_t, col = main_4d(n, d1, d2, ds, coeffs_poly_approx) # main_4d is in hom_solve.jl
df = DataFrame(x=C * h_x, y=C * h_y, z=C * h_z, t=C * h_t, col=col)
df[!, :result] = [f([df.x[i], df.y[i], df.z[i], df.t[i]]) for i in 1:nrow(df)]
df[!, :distance_to_minima] = zeros(nrow(df))
df[!, :steps] = zeros(Int, nrow(df))
df[!, :converged] = falses(nrow(df))

sorted_df = sort(df, :result)

Tracking 2401 paths...   0%|                            |  ETA: 18:14:08



Tracking 2401 paths...   1%|▏                           |  ETA: 2:36:09



Tracking 2401 paths...   1%|▎                           |  ETA: 1:27:24



Tracking 2401 paths...   1%|▍                           |  ETA: 1:08:20



Tracking 2401 paths...   2%|▌                           |  ETA: 0:52:04



Tracking 2401 paths...   2%|▋                           |  ETA: 0:42:53



Tracking 2401 paths...   3%|▊                           |  ETA: 0:35:14



Tracking 2401 paths...   3%|▉                           |  ETA: 0:29:29



Tracking 2401 paths...   4%|█                           |  ETA: 0:25:20



Tracking 2401 paths...   4%|█▏                          |  ETA: 0:22:12



Tracking 2401 paths...   5%|█▎                          |  ETA: 0:19:56



Tracking 2401 paths...   5%|█▍                          |  ETA: 0:18:05



Tracking 2401 paths...   5%|█▌                          |  ETA: 0:16:33



Tracking 2401 paths...  

Row,x,y,z,t,col,result,distance_to_minima,steps,converged
,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Int64,Bool
1,1.3385,-1.01675,1.26518,-0.748234,8,1.57838,0.0,0,false
2,1.41716,-0.977568,1.06025,0.780142,8,2.89905,0.0,0,false
3,-1.05574,-1.03763,-1.3007,1.01636,8,4.9218,0.0,0,false
4,-0.985858,-0.923446,-1.13891,-1.01922,8,6.32282,0.0,0,false
5,-0.924263,-0.406095,-1.12221,-1.09846,8,6.51941,0.0,0,false
6,1.29889,0.425555,1.1192,1.13697,8,7.48803,0.0,0,false
7,-0.750187,1.11526,-1.82005,1.35208,8,7.93971,0.0,0,false
8,1.16724,1.03359,1.2642,-1.26319,8,8.4876,0.0,0,false
9,-0.747374,0.994939,-0.568412,-1.44574,8,11.8743,0.0,0,false


Now we want to compute the distance between the critical points of the 2d function. We want to take the tensor product of coordinates of critical points recovered from the 2D case. We could add a column for minimal distance of 2 first coordinates to original critical point, same for 2nd column. 


In [10]:
using TensorOperations
df4d = kron(df_2d.x, df_2d.y)

225-element Vector{Float64}:
 -0.8521670683086023
 -0.8830201429144513
 -0.6307512285393486
 -0.18006238743013941
 -0.6711628324297435
  0.6307512285394685
  0.8521670683086535
 -0.2536464977730781
  0.7904822855389043
  0.8830201429144948
  ⋮
  0.06902275203178007
 -0.020544538706792036
  0.06402648589619585
  0.07152175040561265
 -5.585477409221794e-15
  0.020544538706777714
  0.05436199951694533
  0.014584465863597709
 -0.06402648589619088

In [5]:


for i in 1:nrow(df)
    println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i], df.z[i], df.t[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)

    distance = norm(x0 - minimizer)

    df.distance_to_minima[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged

    println("Initial point: ", x0)
    println("Minimizer: ", minimizer)
    println("Distance to minima: ", distance)
    println("Number of steps: ", steps)
    println("Converged: ", converged)
    println(summary(res))
end

println(df)

Optimizing for point 1
Iter     Function value   Gradient norm 
     0    -4.386251e-30     6.533105e-15
 * time: 0.020483016967773438
Initial point: [7.379020605317168e-17, -6.508305728350678e-16, -2.1655633393074858e-17, -8.193447953142806e-16]
Minimizer: [7.379020605317168e-17, -6.508305728350678e-16, -2.1655633393074858e-17, -8.193447953142806e-16]
Distance to minima: 0.0
Number of steps: 0
Converged: true
L-BFGS
Optimizing for point 2
Iter     Function value   Gradient norm 
     0     1.920833e+03     7.654431e+02
 * time: 1.2874603271484375e-5
     1     5.686852e+02     3.417921e+02
 * time: 0.26340699195861816
     2     1.290281e+00     3.597028e+00
 * time: 0.263624906539917
     3    -1.295734e-01     1.429806e+00
 * time: 0.2636878490447998
     4    -3.570841e-01     1.032467e+00
 * time: 0.26374101638793945
     5    -4.213354e-01     4.185087e-01
 * time: 0.2637908458709717
     6    -4.308140e-01     3.720770e-02
 * time: 0.263840913772583
     7    -4.309276e-01     1

Excessive output truncated after 524291 bytes.

Initial point: [3.5355339059327378, 1.6368770376893263e-16, 3.5355339059327378, 

In [9]:
println(df)